# データサイエンス50問 - Polars

In [1]:
!pip install polars
!pip install japanize_matplotlib
!pip install --user--force-reinstall pyarrow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 783.6/783.6 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 MB 51.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [polars]2m1/2 [polars]

[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 34.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for japanize_matplotlib: filename=japanize_matplotlib-1.1.3-py3-none-any.whl size=4120306 sha256=15a952f4d7c52efe7efff16c72d2245bef996c7ba5255b4b769af278bcaae439
  Stored in directory: /home/codespace/.cache/pip/wheels/c1/f7/9b/418f19a7b9340fc16e071e89efc379aca68d40238b258df53d
Successfully built japanize_matplotlib

[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: pyt

In [6]:
"""
Based on: Data Processing 100 Knocks by The Japan DataScientist Society
Original Repository: https://github.com/The-Japan-DataScientist-Society/100knocks-preprocess
"""
import os
import polars as pl
from matplotlib import pyplot as plt
# plt.rcParams['font.family'] = ['IPAexGothic']
import japanize_matplotlib

In [7]:
pl.__version__

'1.35.2'

In [8]:
dtypes = {
    'customer_id': str,
    'gender_cd': str,
    'postal_cd': str,
    'application_store_cd': str,
    'status_cd': str,
    'category_major_cd': str,
    'category_medium_cd': str,
    'category_small_cd': str,
    'product_cd': str,
    'store_cd': str,
    'prefecture_cd': str,
    'tel_no': str,
    'postal_cd': str,
    'street': str,
    'application_date': str,
    'birth_day': pl.Date
}

df_customer = pl.read_csv('./data/customer.csv', schema_overrides=dtypes, encoding='utf-8-sig')
df_receipt = pl.read_csv('./data/receipt.csv', schema_overrides=dtypes, encoding='utf-8-sig')
df_product = pl.read_csv('./data/product.csv', schema_overrides=dtypes, encoding='utf-8-sig')
df_category = pl.read_csv('./data/category.csv', schema_overrides=dtypes, encoding='utf-8-sig')
df_store = pl.read_csv('./data/store.csv', schema_overrides=dtypes, encoding='utf-8-sig')
df_geocode = pl.read_csv('./data/geocode.csv', schema_overrides=dtypes, encoding='utf-8-sig')

### ここからの問題にPolarsを用いて取り組むこと

問題1

どのようなデータを保有しているか目視確認できるように、レシート明細データ（df_receipt）から全項目の先頭5件を表示せよ。

<img src="https://i.gyazo.com/6531a5f636245f384ace6400d8e40861.png" width=70%>

In [9]:
df_receipt.head(5)


sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
i64,i64,str,i64,i64,str,str,i64,i64
20181103,1541203200,"""S14006""",112,1,"""CS006214000001""","""P070305012""",1,158
20181118,1542499200,"""S13008""",1132,2,"""CS008415000097""","""P070701017""",1,81
20170712,1499817600,"""S14028""",1102,1,"""CS028414000014""","""P060101005""",1,170
20190205,1549324800,"""S14042""",1132,1,"""ZZ000000000000""","""P050301001""",1,25
20180821,1534809600,"""S14025""",1102,2,"""CS025415000050""","""P060102007""",1,90


問題2

レシート明細データ（df_receipt）から売上年月日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、5件表示せよ。

<img src="https://i.gyazo.com/1445da71429b477b0a898bc7039666b9.png" width="400px">

In [23]:
df_receipt[['sales_ymd', 'customer_id', 'product_cd', 'amount']].head(5)



sales_ymd,customer_id,product_cd,amount
i64,str,str,i64
20181103,"""CS006214000001""","""P070305012""",158
20181118,"""CS008415000097""","""P070701017""",81
20170712,"""CS028414000014""","""P060101005""",170
20190205,"""ZZ000000000000""","""P050301001""",25
20180821,"""CS025415000050""","""P060102007""",90


In [27]:
df_receipt[['sales_ymd', 'customer_id', 'product_cd', 'amount']].head(5)

sales_ymd,customer_id,product_cd,amount
i64,str,str,i64
20181103,"""CS006214000001""","""P070305012""",158
20181118,"""CS008415000097""","""P070701017""",81
20170712,"""CS028414000014""","""P060101005""",170
20190205,"""ZZ000000000000""","""P050301001""",25
20180821,"""CS025415000050""","""P060102007""",90


問題3

レシート明細データ（df_receipt）から売上年月日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、5件表示せよ。ただし、sales_ymdという項目名については、sales_dateに変更して抽出すること。


<img src="https://i.gyazo.com/47f0bb5d9642b0633a0ea411c3c94882.png"  width="400px">

In [ ]:

df_receipt.select([
    pl.col("sales_ymd").alias("sales_date"),
    pl.col("customer_id"),
    pl.col("product_cd"),
    pl.col("amount")
]).head(5)

sales_date,customer_id,product_cd,amount
i64,str,str,i64
20181103,"""CS006214000001""","""P070305012""",158
20181118,"""CS008415000097""","""P070701017""",81
20170712,"""CS028414000014""","""P060101005""",170
20190205,"""ZZ000000000000""","""P050301001""",25
20180821,"""CS025415000050""","""P060102007""",90


問題4

レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出し、先頭の5件を表示せよ。
> - 顧客ID（customer_id）が"CS018205000001"

<img src="https://i.gyazo.com/7ed7be141e0a4ae0927f921892869ca2.png"  width="400px">

In [36]:
(df_receipt
.select("sales_ymd", "customer_id", "product_cd", "amount")
.filter(pl.col("customer_id")=="CS018205000001")
.head(5)
)


sales_ymd,customer_id,product_cd,amount
i64,str,str,i64
20180911,"""CS018205000001""","""P071401012""",2200
20180414,"""CS018205000001""","""P060104007""",600
20170614,"""CS018205000001""","""P050206001""",990
20170614,"""CS018205000001""","""P060702015""",108
20190216,"""CS018205000001""","""P071005024""",102


問題5

レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上

<img src="https://i.gyazo.com/7ad9b149fcd219db6829838881013a79.png"  width="400px">

In [38]:
(df_receipt
    .select(
        pl.col("sales_ymd").alias("sales_date"),
        "customer_id",
        "product_cd",
        "amount")
    .filter(pl.col("customer_id") == "CS018205000001")
    .filter(pl.col("amount") > 1000)
    .head(5)
)


sales_date,customer_id,product_cd,amount
i64,str,str,i64
20180911,"""CS018205000001""","""P071401012""",2200
20190226,"""CS018205000001""","""P071401020""",2200
20180911,"""CS018205000001""","""P071401005""",1100


問題6

店舗データ（df_store）から、住所 (address) に"横浜市"が含まれるものだけを抽出し、先頭の5件を表示せよ。

<img src="https://i.gyazo.com/26eb74c390635a6704f7cc41267e79a3.png" width=80%>

In [47]:
(df_store
    .filter(pl.col("address").str.contains('横浜市',))
    .head(5)
 )

store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
str,str,str,str,str,str,str,f64,f64,f64
"""S14010""","""菊名店""","""14""","""神奈川県""","""神奈川県横浜市港北区菊名一丁目""","""カナガワケンヨコハマシコウホククキクナイッチョウメ""","""045-123-4032""",139.6326,35.50049,1732.0
"""S14033""","""阿久和店""","""14""","""神奈川県""","""神奈川県横浜市瀬谷区阿久和西一丁目""","""カナガワケンヨコハマシセヤクアクワニシイッチョウメ""","""045-123-4043""",139.4961,35.45918,1495.0
"""S14040""","""長津田店""","""14""","""神奈川県""","""神奈川県横浜市緑区長津田みなみ台五丁目""","""カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ""","""045-123-4046""",139.4994,35.52398,1548.0
"""S14050""","""阿久和西店""","""14""","""神奈川県""","""神奈川県横浜市瀬谷区阿久和西一丁目""","""カナガワケンヨコハマシセヤクアクワニシイッチョウメ""","""045-123-4053""",139.4961,35.45918,1830.0
"""S14028""","""二ツ橋店""","""14""","""神奈川県""","""神奈川県横浜市瀬谷区二ツ橋町""","""カナガワケンヨコハマシセヤクフタツバシチョウ""","""045-123-4042""",139.4963,35.46304,1574.0


問題7

顧客データ（df_customer）を生年月日（birth_day）で高齢順にソートし、先頭から5件表示せよ。

<img src="https://i.gyazo.com/0842c9168b3021da451980dcd83c1ba9.png" width=80%>

In [48]:
(df_customer
    .sort("birth_day")
    .head(5)
)


customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
str,str,str,str,date,i64,str,str,str,str,str
"""CS003813000014""","""村山 菜々美""","""1""","""女性""",1928-11-26,90,"""182-0007""","""東京都調布市菊野台**********""","""S13003""","""20160214""","""0-00000000-0"""
"""CS026813000004""","""吉村 朝陽""","""1""","""女性""",1928-12-14,90,"""251-0043""","""神奈川県藤沢市辻堂元町**********""","""S14026""","""20150723""","""0-00000000-0"""
"""CS018811000003""","""熊沢 美里""","""1""","""女性""",1929-01-07,90,"""204-0004""","""東京都清瀬市野塩**********""","""S13018""","""20150403""","""0-00000000-0"""
"""CS027803000004""","""内村 拓郎""","""0""","""男性""",1929-01-12,90,"""251-0031""","""神奈川県藤沢市鵠沼藤が谷**********""","""S14027""","""20151227""","""0-00000000-0"""
"""CS013801000003""","""天野 拓郎""","""0""","""男性""",1929-01-15,90,"""274-0824""","""千葉県船橋市前原東**********""","""S12013""","""20160120""","""0-00000000-0"""


問題8

顧客データ（df_customer）を生年月日（birth_day）で若い順となるようにソートし、先頭から5件表示せよ。

<img src="https://i.gyazo.com/285d0a57cd786e3bf0947ce2f76af125.png" width=80%>

In [49]:
(df_customer
    .sort("birth_day",descending=True)
    .head(5)
)

customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
str,str,str,str,date,i64,str,str,str,str,str
"""CS035114000004""","""大村 美里""","""1""","""女性""",2007-11-25,11,"""156-0053""","""東京都世田谷区桜**********""","""S13035""","""20150619""","""6-20091205-6"""
"""CS022103000002""","""福山 はじめ""","""9""","""不明""",2007-10-02,11,"""249-0006""","""神奈川県逗子市逗子**********""","""S14022""","""20160909""","""0-00000000-0"""
"""CS002113000009""","""柴田 真悠子""","""1""","""女性""",2007-09-17,11,"""184-0014""","""東京都小金井市貫井南町**********""","""S13002""","""20160304""","""0-00000000-0"""
"""CS004115000014""","""松井 京子""","""1""","""女性""",2007-08-09,11,"""165-0031""","""東京都中野区上鷺宮**********""","""S13004""","""20161120""","""1-20081231-1"""
"""CS002114000010""","""山内 遥""","""1""","""女性""",2007-06-03,11,"""184-0015""","""東京都小金井市貫井北町**********""","""S13002""","""20160920""","""6-20100510-1"""


問題9

レシート明細データ（df_receipt）に対し、件数をカウントせよ。

<img src="https://i.gyazo.com/bb5dbb81d7aaf9783efbe69c12f8676c.png">

In [50]:
len(df_receipt)

104681

問題10

レシート明細データ（df_receipt）の顧客ID（customer_id）に対し、ユニークな件数をカウントせよ。

<img src="https://i.gyazo.com/e59119c262ac028e8f59cb40fd1fac09.png">

In [54]:
len(df_receipt
    .select("customer_id")
    .unique()
)

8307

問題11

顧客データ（df_customer）の生年月日（birth_day）は日付型でデータを保有している。これをYYYYMMDD形式の文字列に変換し、顧客ID（customer_id）とともに5件表示せよ。
> - YYYYMMDD形式とは例えば20240401という文字列（str型）のこと

<img src="https://i.gyazo.com/90dafb4cca271b0d2414654ab2ac6038.png"  width="200px">

In [ ]:
(df_customer)

問題12

顧客データ（df_customer）の申し込み日（application_date）はYYYYMMDD形式の**文字列型**でデータを保有している。これを日付型（date型）に変換し、顧客ID（customer_id）とともに5件表示せよ。

<img src="https://i.gyazo.com/a3c33845281c0f1fbe1484ef94b28c98.png"  width="200px">

問題13

レシート明細データ（df_receipt）の売上日（sales_ymd）はYYYYMMDD形式の**数値型**でデータを保有している。これを日付型に変換し、レシート番号（receipt_no）、レシートサブ番号（receipt_sub_no）とともに5件表示せよ。

> - ヒント：数値型から文字列型に変換して、その後に日付型に変換する

<img src="https://i.gyazo.com/2319373413976bf9fe1a84d48e1919b0.png"  width="200px">

問題14

レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）と売上数量（quantity）を合計せよ。また、店舗コード（store_cd）で並べ替えて最初の5件表示せよ。

<img src="https://i.gyazo.com/faa30f87295d53ede6c8c639ebfe046e.png" width="200px">

問題15

レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上年月日（sales_ymd）の5件を表示せよ。

<img src="https://i.gyazo.com/518416a9eec4a612faf935ef451edd22.png" width="200px">

問題16

レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとの最も古い売上年月日（sales_ymd）の5件を表示せよ。

<img src="https://i.gyazo.com/423f835a4545d63da7181007bfc74ccd.png" width="200px">

問題17

レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上年月日（sales_ymd）と古い売上年月日を示し、両者が異なるデータを5件表示せよ。ただし、顧客ID（customer_id）を昇順で並べ替えて最初の5件表示すること。

<img src="https://i.gyazo.com/acc87ff955fbb8f37b93f0e347176f2f.png"  width="300px">

問題18

レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を示し、降順でTOP5を表示せよ。

<img src="https://i.gyazo.com/9cc41ee9e7581e4b1bb7cd5958afaef9.png" width="200px">

問題19

レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を示し、330以上のものを抽出せよ。また、店舗コード（store_cd）を昇順で並べ替えて最初の5件表示せよ。

<img src="https://i.gyazo.com/8d02f74c9585350133eb158dcd50a76e.png" width="200px">

問題20

レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して、その全顧客の平均を求めよ。

<img src="https://i.gyazo.com/a5af1c5474898c30242b8880ca5761df.png">

問題21（難易度：★★☆）

顧客ID（customer_id）が"Z"から始まるものは非会員を表すため、除外して以下の計算をすること。レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計した後、全顧客の平均売上金額（avg_amountに名前を変える）を示す項目を追加し、その平均売上金額（avg_amount）以上に買い物をしている顧客を抽出し、顧客ID（customer_id）を昇順で並べ替えて最初の5件表示せよ。

> - ヒント：avg_amountは全顧客の合計売上金額の平均値なので全ての行で同じ値になる（赤枠部分）。それと各顧客の合計売上金額を比較する。

<img src="https://i.gyazo.com/e5c89880d5325b1f24268106b6001cfb.png" width="300px">

問題22

店舗コード（store_cd）をキーとして、レシート明細データ（df_receipt）と店舗データ（df_store）を内部結合し、レシート明細データ（df_receipt）の全項目と店舗データ（df_store）の店舗名（store_name）を5件表示せよ。

> ※ 内部結合とは、両データに存在するキーの値で結合する方法（つまり、片方のデータにしか存在しないキーの値は結合後のデータに含まれない）

<img src="https://i.gyazo.com/919d7cc43fd65ebe75082a13aeab5fd8.png" width=70%>

問題23

カテゴリ小区分コード（category_small_cd）をキーとして、商品データ（df_product）とカテゴリデータ（df_category）を内部結合し、商品データ（df_product）の全項目とカテゴリデータ（df_category）のカテゴリ小区分名（category_small_name）を5件表示せよ。

<img src="https://i.gyazo.com/3042388dabf489d75fcbc09f52a49a33.png" width=70%>

問題24（難易度：★★☆）

顧客ID（customer_id）が"Z"から始まるものは非会員を表すため、除外して以下の計算をすること。また、顧客の性別コード（gender_cd）が女性（"1"）であるものを対象とする。顧客データ（df_customer）とレシート明細データ（df_receipt）を顧客コード（customer_id）で結合し、顧客ごとの売上金額合計を求め、顧客コードの順番に、性別（gender）と売上金額合計を10件表示せよ。ただし、売上実績がない顧客については売上金額を0として表示させること。

> - 注意：gender_cdは数値型ではなく文字列型

<img src="https://i.gyazo.com/a320b81c661983d7abd7c6f5d5160dc0.png" width="250px">

問題25（難易度：★★★）

売上日数の多い顧客、もしくは売上金額合計の大きい顧客を抽出したい。そこで、レシート明細データ（df_receipt）から、売上日数（sales_ymd）が多い顧客の上位20件を抽出したデータと、売上金額（amount）の合計が大きい顧客の上位20件を抽出したデータをそれぞれ作成し、その2つを外部結合し、先頭の5件を表示せよ。ただし、非会員（顧客IDが"Z"から始まるもの）は除外すること。

※外部結合とは、片方のデータにしか存在しないキーの値も結合後のデータに含める方法

<img src="https://i.gyazo.com/e673852056870ab7bd667825155db835.png" width="350px">

問題26

レシート明細データ（df_receipt）の売上金額（amount）を日付（sales_ymd）ごとに合計し、各日付のデータに対し、前回、前々回、3回前に売上があった日のデータを結合せよ。そして結果を5件表示せよ。

<img src="https://i.gyazo.com/732d8f115ee54b3fecbd7036448b4e24.png" width="350px">

問題27（難易度：★★☆）

レシート明細データ（df_receipt）の売上金額（amount）を日付（sales_ymd）ごとに合計し、前回売上があった日からの売上金額増減を計算せよ。そして結果を5件表示せよ。

<img src="https://i.gyazo.com/c05817bec9703ae09cfb1b0ee3d06d61.png" width="250px">

問題28（難易度：★★★）

レシート明細データ（df_receipt）と顧客データ（df_customer）を顧客コード（customer_id）で結合し、性別（gender）と年代（ageが58であれば年代は50と計算する）ごとに売上金額（amount）を合計した売上サマリデータを作成せよ。

ただし、縦に年代、横に性別（性別の並び順は指定しなくて良い）のクロス集計表とし、年代は10歳ごとの階級とすること。

ヒント（年代について）：58//10 を実行すると5となることを利用して求めよ。

<img src="https://i.gyazo.com/5c59456ba027ecc27fdd539faeb09af0.jpg" width="250px">

問題29（省略OK）

レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計の上、売上金額合計に対して2,000円以下を0、2,000円より大きい金額を1に二値化した項目を作成し、顧客ID、売上金額合計とともに5件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

<img src="https://i.gyazo.com/3eb56acb21d441c7e1478fddd2fe7d26.png" width="250px">

問題30（省略OK）

顧客データ（df_customer）の住所（address）は、埼玉県、千葉県、東京都、神奈川県のいずれかとなっている。都道府県毎にコード値（埼玉県を11、千葉県を12、東京都を13、神奈川県を14）を示す項目（prefecture）を作成し、顧客ID、住所とともに5件表示せよ。

<img src="https://i.gyazo.com/d22630dd7f8202984e2d600f9323a6c9.png" width="350px">

問題31（省略OK）

顧客データ（df_customer）の年齢（age）をもとに10歳刻みで年代を算出し、顧客ID（customer_id）、生年月日（birth_day）とともに5件表示せよ。ただし、60歳以上は全て60歳代とすること。年代を表すカテゴリ名は任意とする。

<img src="https://i.gyazo.com/167c7c7aeb98b15185296a262b9687bb.png" width="250px">

問題32（難易度：★★★）

レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計した項目（total_amount）を作成し、その売上金額合計の全顧客の平均との差を示す項目（amount_diff）を作成し、顧客IDの順番に5件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

<img src="https://i.gyazo.com/2dadd138ef43f10e381b1989176b655b.png" width="300px">

問題33

商品データ（df_product）の単価（unit_price）と原価（unit_cost）から各商品の利益額を示す項目（unit_profit）を追加し、結果を5
件表示せよ。

<img src="https://i.gyazo.com/0f76d3c2a7b6ea1d294651268ef4e120.png" width=70%>

問題34

商品データ（df_product）の単価（unit_price）と原価（unit_cost）から、各商品の利益率を求め、その全体平均を算出せよ。

<img src="https://i.gyazo.com/27d324f153382527682b45e75080553f.png">

問題35（難易度：★★☆）

商品データ（df_product）の各商品について、利益率が30%となるような新たな単価（new_price）を示す項目を追加せよ。ただし、1円未満は切り捨てること。そして、新たな単価（new_price）による新たな利益率（new_profit_rate）を示す項目を追加して、およそ30％付近であることを5件表示して確認せよ。

> - ヒント1：利益率30％となる単価は「コスト / 0.7」で求める
> - ヒント2：切り捨ては「.floor()」を用いる

<img src="https://i.gyazo.com/2a7a0fa1ff01885ab017e983b0517411.png" width=70%>

問題36

商品データ（df_product）の各商品について、消費税率10％の税込み金額を示す項目（tax_price）を追加せよ。1円未満の端数は切り捨てとし、結果を5件表示せよ。

<img src="https://i.gyazo.com/c649004624b96510b9342f0c955b8516.png"  width="250px">

問題37（難易度：★★☆）

レシート明細データ（df_receipt）と商品データ（df_product）を商品コード（product_cd）で内部結合し、顧客毎に全商品の売上金額合計（total_amount）と、カテゴリ大区分コード（category_major_cd）が"07"（瓶詰缶詰）の売上金額合計（total_amount_07）を示す項目を追加せよ。また、その両者の比率を示す項目（sales_rate）も追加し、顧客コードの順番に5件表示せよ。

<img src="https://i.gyazo.com/ec83d3cf8f69553b8a8ee18565e33b27.png" width="350px">

問題38（難易度：★★☆）

レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、顧客データ（df_customer）の会員申込日（application_date）からの経過日数を示す項目（elapsed_days）を追加し、顧客ID（customer_id）、売上日、会員申込日とともにせよ。
ただし、経過日数は顧客ごとに「最新の売上日」と「会員申込日」との差で計算すること。

また、顧客ID順で最初の5件を表示せよ。

> - ヒント1：まずは各顧客の最新の売上日（売上日の最大値）を示す項目（last_ymd）を計算する。
> - ヒント2：sales_ymdは数値、application_dateは文字列でデータを保持している点に注意。そして経過日数はdate型同士で計算すること。

<img src="https://i.gyazo.com/3849c05f2c5135f1427c0395789e5ee6.png" width="400px">

問題39

顧客データ（df_customer）からランダムに1%のデータを抽出し、先頭から5件表示せよ。

<img src="https://i.gyazo.com/fba779ef5d6e29e0aeaf9717abf1c503.png" width=80%>

問題40

顧客データ（df_customer）からランダムに5件のデータを表示せよ。

<img src="https://i.gyazo.com/9d1582a46305ccf4b2d0ba130c7023fe.png" width=80%>

問題41

顧客データ（df_customer）から性別（gender）ごとの件数を表示せよ。

<img src="https://i.gyazo.com/ac3ad3f012090c4f793a2dadfa772ee9.png" width="150px">

問題42（難易度：★★☆）

レシート明細データ（df_receipt）の売上金額を顧客単位に合計し、合計した売上金額の外れ値を抽出せよ。なお、外れ値は、各顧客の売上金額合計が、全顧客の売上金合計の平均から3σを超えて離れたものとする。

> - ヒント1：絶対値「.abs()」を用いるとシンプルになる。
> - σは「pl.col("amount).std()」で計算できる。

<img src="https://i.gyazo.com/14db3c4f811edf641d1901619fbcac88.png">

問題43

商品データ（df_product）の各項目に対し、欠損数を確認せよ。

<img src="https://i.gyazo.com/11ba9e841b970b8faaaa5639e6841d34.png" width=70%>

問題44

商品データ（df_product）のいずれかの項目に欠損が発生しているレコードを全て削除した商品データを作成せよ。なお、削除前後の件数をそれぞれ表示して、一つ前の問題で確認した件数だけ減少していることも確認すること。

<img src="https://i.gyazo.com/fe6905b13c8ad447f22972bb4215a29c.png">

問題45（難易度：★★☆）

レシート明細データ（df_receipt）の各顧客に対して、全期間の売上金額に占める2019年売上金額の割合を示す項目（amount_rate）を追加せよ。顧客ID順に5件示せ。

> - ヒント1：全期間の売上金額と2019年の売上金額を顧客単位でそれぞれ合計してから計算すること。
> - ヒント2：df_receiptのsales_ymdはYYYYMMDD形式の数値でデータを保有している点に注意。

<img src="https://i.gyazo.com/05c1056cced409ab8301995bd1f22dc9.png"  width="400px">

### ここからはグラフ表示に関する問題です。

問題46

顧客データ（df_customer）から性別（gender）ごとの顧客数を棒グラフで示せ。ただし、顧客数の多い順番で性別を並べ替えて表示すること。横軸の目盛に性別が表示されるように気を付ける。

<img src="https://i.gyazo.com/29c5b71b89c0a11e380cd3adf7ffade2.png" width="300px">

問題47

レシート明細データ（df_receipt）の顧客（customer_id）ごとの売上金額（amount）を棒グラフで示せ。ただし、顧客IDが"Z"から始まるものは非会員を表すため、除外して計算し、売上金額合計の大きい上位10顧客を表示すること。

また、グラフのタイトル、X軸、Y軸のラベル、グラフサイズは以下とすること。
- タイトル：売上合計上位10顧客
- X軸ラベル：顧客ID
- Y軸ラベル：合計金額
- グラフサイズ：8x3


<img src="https://i.gyazo.com/76bdec3e9ff0f6f4bddd22dd55cd76e6.png" width="400px">

問題48（難易度：★★☆）

レシート明細データ（df_receipt）において、顧客（customer_id）ごとの年ごとの売上金額（amount）の合計を棒グラフで示せ。グラフには10件ほどの顧客を示せば良い。ただし、年ごとの売上合計は個々の顧客において積み上げグラフとして示すこと。 

> - ヒント1：まずは売上日（sales_ymd: 数値型）をもとに売上年を表す項目（sales_year）を作成する
> - ヒント2：顧客ごとの売上年ごとの売上合計金額を集計する
> - ヒント3：行を顧客ID、列を売上年、値を売上合計とする表に変換する

<img src="https://i.gyazo.com/064b66492184090c3a7090896a5f07dd.png" width="400px">
<img src="https://i.gyazo.com/c5d1103e1cd5cbcf1c043abd4835f4d9.png" width="400px">

問題49（難易度：★★☆）

レシート明細データ（df_receipt）の売上日（sales_ymd）ごとの売上金額（amount）の合計を折れ線グラフで示せ。ただし、売上日が2019年1月1日から2019年3月31日までのデータを対象とする。グラフのマーカーとラインは以下とすること。
> - ライン：点線（--）
> - マーカー：丸（o）

> ヒント：sales_ymdは数値型なのでこのままでは下のグラフのようにはならない。型の変換が必要。

<img src="https://i.gyazo.com/6e4aae9f376eb3f9a0ea2d3882c598c8.png" width="400px">

問題50（難易度：★★★）

レシート明細データ（df_receipt）の売上日（sales_ymd）ごとの売上金額（amount）と売上数量（quantity）をそれぞれ折れ線グラフで示せ。ただし、金額と数量は単位が異なるため、左の縦軸を合計金額、右の縦軸を合計数量としてグラフにすること。対象は、売上日が2019年1月1日から2019年1月31日までのデータをとする。グラフは以下の設定にしたがうこと。
> - ライン：点線（--）
> - マーカー：丸（o）
> - 凡例の位置：売上金額は左上、売上数量は右上

<img src="https://i.gyazo.com/8884d67db2037e80d16d2e86e7415088.png" width="450px">